In [1]:
from awrams.simulation.server import Server
from awrams.models import awral

from awrams.utils import extents
from awrams.utils import datetools

In [2]:
def change_path_to_forcing(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    FORCING = {
        'tmin': ('temp_min*','temp_min_day',data_path),
        'tmax': ('temp_max*','temp_max_day',data_path),
        'precip': ('rain_day*','rain_day',data_path),
        'solar': ('solar*','solar_exposure_day',data_path)
    }
    for k,v in FORCING.items():
        imap.mapping[k+'_f'] = nodes.forcing_from_ncfiles(v[2],v[0],v[1])


In [3]:
imap = awral.get_default_mapping()
change_path_to_forcing(imap)

### create model output map and add nodes for saving some model outputs to netcdf files

In [4]:
def build_output_mapping():
    from awrams.utils.nodegraph import nodes,graph
    from awrams.models.awral.template import DEFAULT_TEMPLATE

    ### populate output map with all model outputs
    output_map = awral.get_output_nodes(DEFAULT_TEMPLATE)

    outpath = './_results/'

    output_map.mapping['s0_ncsave'] = nodes.write_to_annual_ncfile(outpath,'s0')
    output_map.mapping['ss_ncsave'] = nodes.write_to_annual_ncfile(outpath,'ss')
    output_map.mapping['sd_ncsave'] = nodes.write_to_annual_ncfile(outpath,'sd')
    output_map.mapping['qtot_ncsave'] = nodes.write_to_annual_ncfile(outpath,'qtot')
    output_map.mapping['etot_ncsave'] = nodes.write_to_annual_ncfile(outpath,'etot')

    return graph.OutputGraph(output_map.mapping)


In [5]:
omap = build_output_mapping()

In [6]:
period = datetools.dates('dec 2010 - jan 2011')
extent = extents.from_boundary_offset(200,200,250,250)

In [7]:
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:21:49,578 INFO Getting I/O dataspecs...
2016-10-13 04:21:50,217 INFO Initialising output files...
2016-10-13 04:21:50,398 INFO Building buffers...
2016-10-13 04:21:51,509 INFO Running simulation...


worker pid: 9714
worker pid: 9713


2016-10-13 04:21:52,648 INFO completed 25.00%
2016-10-13 04:21:51,549 INFO reader pid: 9732
2016-10-13 04:21:51,550 INFO writer pid: 9736
2016-10-13 04:21:52,788 INFO completed 50.00%
2016-10-13 04:21:52,858 INFO completed 75.00%
2016-10-13 04:21:52,917 INFO completed 100.00%
2016-10-13 04:21:52,917 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:21:53,025 INFO completed 25.00%
2016-10-13 04:21:53,134 INFO completed 50.00%
2016-10-13 04:21:53,222 INFO completed 75.00%
2016-10-13 04:21:53,284 INFO completed 100.00%
2016-10-13 04:21:53,299 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:21:55,728 INFO elapsed time: 6.17


worker pid: 9717
worker pid: 9720
reader pid: 9732


### initialise states from dict mapping of numpy arrays

In [8]:
def initial_states_from_dict(imap,period,extent):
    from awrams.utils.io.data_mapping import SplitFileManager
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    node_names = {'mleaf_dr': 'init_mleaf_hrudr',
                  'mleaf_sr': 'init_mleaf_hrusr',
                  's0_dr': 'init_s0_hrudr',
                  's0_sr': 'init_s0_hrusr',
                  'ss_dr': 'init_ss_hrudr',
                  'ss_sr': 'init_ss_hrusr',
                  'sd_dr': 'init_sd_hrudr',
                  'sd_sr': 'init_sd_hrusr',
                  'sg_bal': 'init_sg',
                  'sr_bal': 'init_sr'}

    data_map = {}
    period = [period[0] - 1]
    
    for k in node_names:
        sfm = SplitFileManager.open_existing(data_path,k+'*nc',k)
        data_map[node_names[k]] = sfm.get_data(period,extent)

    nodes.init_states_from_dict(imap,data_map,extent)


In [9]:
initial_states_from_dict(imap,period,extent)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:21:56,111 INFO Getting I/O dataspecs...
2016-10-13 04:21:56,750 INFO Initialising output files...
2016-10-13 04:21:56,886 INFO Building buffers...
2016-10-13 04:21:57,003 INFO Running simulation...


worker pid: 9808
worker pid: 9809
worker pid: 9814


2016-10-13 04:21:58,081 INFO completed 25.00%
2016-10-13 04:21:57,037 INFO writer pid: 9831
2016-10-13 04:21:57,042 INFO reader pid: 9830
2016-10-13 04:21:58,187 INFO completed 50.00%
2016-10-13 04:21:58,297 INFO completed 75.00%
2016-10-13 04:21:58,357 INFO completed 100.00%
2016-10-13 04:21:58,358 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:21:58,467 INFO completed 25.00%
2016-10-13 04:21:58,560 INFO completed 50.00%
2016-10-13 04:21:58,669 INFO completed 75.00%
2016-10-13 04:21:58,739 INFO completed 100.00%
2016-10-13 04:21:58,748 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:22:00,354 INFO elapsed time: 4.26


worker pid: 9823
reader pid: 9830


### initialise states from netcdf files

In [10]:
def get_initial_states(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    mapping = imap.mapping

    mapping['init_sr'] = nodes.init_state_from_ncfile(data_path,'sr_bal*','sr_bal')
    mapping['init_sg'] = nodes.init_state_from_ncfile(data_path,'sg_bal*','sg_bal')

    HRU = {'_hrusr':'_sr','_hrudr':'_dr'}
    for hru in ('_hrusr','_hrudr'):
        for state in ["s0","ss","sd",'mleaf']:
            mapping['init_'+state+hru] = nodes.init_state_from_ncfile(data_path,state+HRU[hru]+'*',state+HRU[hru])


In [11]:
initial_states_from_dict(imap,period,extent)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:22:00,673 INFO Getting I/O dataspecs...
2016-10-13 04:22:01,308 INFO Initialising output files...
2016-10-13 04:22:01,499 INFO Building buffers...
2016-10-13 04:22:01,619 INFO Running simulation...


worker pid: 9907
worker pid: 9908
worker pid: 9913


2016-10-13 04:22:02,669 INFO completed 25.00%
2016-10-13 04:22:01,653 INFO writer pid: 9933
2016-10-13 04:22:01,657 INFO reader pid: 9929
2016-10-13 04:22:02,802 INFO completed 50.00%
2016-10-13 04:22:02,886 INFO completed 75.00%
2016-10-13 04:22:02,971 INFO completed 100.00%
2016-10-13 04:22:02,994 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:22:03,061 INFO completed 25.00%
2016-10-13 04:22:03,173 INFO completed 50.00%
2016-10-13 04:22:03,246 INFO completed 75.00%
2016-10-13 04:22:03,329 INFO completed 100.00%
2016-10-13 04:22:03,330 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:22:05,898 INFO elapsed time: 5.24


worker pid: 9922
reader pid: 9929


### fill gaps in forcing data with climatology

In [12]:
def insert_climatology(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    imap.mapping['precip_f'] = nodes.forcing_gap_filler(data_path,'rain_day*','rain_day',join(data_path,'climatology_daily_rain_day.nc'))
    imap.mapping['tmax_f'] = nodes.forcing_gap_filler(data_path,'temp_max*','temp_max_day',join(data_path,'climatology_daily_temp_max_day.nc'))
    imap.mapping['tmin_f'] = nodes.forcing_gap_filler(data_path,'temp_min*','temp_min_day',join(data_path,'climatology_daily_temp_min_day.nc'))
    imap.mapping['solar_f'] = nodes.forcing_gap_filler(data_path,'solar*','solar_exposure_day',join(data_path,'climatology_daily_solar_exposure_day.nc'))
    

In [13]:
insert_climatology(imap)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:22:05,951 INFO Getting I/O dataspecs...
2016-10-13 04:22:06,629 INFO Initialising output files...
2016-10-13 04:22:06,760 INFO Building buffers...
2016-10-13 04:22:06,879 INFO Running simulation...


worker pid: 10002
worker pid: 10003
worker pid: 10008
worker pid: 10017


2016-10-13 04:22:08,041 INFO completed 25.00%
2016-10-13 04:22:06,918 INFO writer pid: 10030
2016-10-13 04:22:06,920 INFO reader pid: 10027
2016-10-13 04:22:08,187 INFO completed 50.00%
2016-10-13 04:22:08,263 INFO completed 75.00%
2016-10-13 04:22:08,345 INFO completed 100.00%
2016-10-13 04:22:08,346 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:22:08,433 INFO completed 25.00%
2016-10-13 04:22:08,526 INFO completed 50.00%
2016-10-13 04:22:08,647 INFO completed 75.00%
2016-10-13 04:22:08,697 INFO completed 100.00%
2016-10-13 04:22:08,703 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:22:10,337 INFO elapsed time: 4.40


reader pid: 10027
